## Project: Predicting Stock Market
## Part 1: Data preparation, model estimation
**Data:** S&P500 index valuation, New York Stock Exchange (NYSE), 1950 to 2015  
**DataQuest Lesson:** https://app.dataquest.io/c/11/m/65/guided-project%3A-predicting-the-stock-market/1/the-dataset?path=2&slug=data-scientist&version=1.2  
**Source:** https://github.com/NickyThreeNames/DataquestGuidedProjects/blob/master/Guided%20Project-%20Predicting%20the%20stock%20market/sphist.csv


#### Useful links

Inspiration notebooks:
* https://medium.com/shiyan-boxer/s-p-500-stock-price-prediction-using-machine-learning-and-deep-learning-328b1839d1b6
* https://medium.com/mlearning-ai/predict-sp500-stock-price-with-python-machine-learning-sentiment-analysis-a296dc276353
* https://medium.com/p/59b06de25357
* https://iopscience.iop.org/article/10.1088/1742-6596/1366/1/012130
* https://towardsdatascience.com/stock-market-predictions-with-rnn-using-daily-market-variables-6f928c867fd2
* https://www.kaggle.com/code/samaxtech/predicting-s-p500-index-linearreg-randomforests
* https://www.kaggle.com/code/janiobachmann/s-p-500-time-series-forecasting-with-prophet

Feature Engineering:
* https://towardsdatascience.com/one-hot-encoding-scikit-vs-pandas-2133775567b8
* https://towardsdatascience.com/using-columntransformer-to-combine-data-processing-steps-af383f7d5260
* https://www.kaggle.com/code/samaxtech/predicting-s-p500-index-linearreg-randomforests
* https://www.alpharithms.com/predicting-stock-prices-with-linear-regression-214618/
* https://towardsdatascience.com/3-basic-steps-of-stock-market-analysis-in-python-917787012143
* https://tradewithpython.com/generating-buy-sell-signals-using-python
* https://not-satoshi.com/course-binance-api-trading-bot-sma-macd-and-bollinger-bands/
* https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html

Correlation for non-numeric data (Dython library):  

Variable standarization:  

Variable selection:  

Optimal Binning and WoE transformation for continuous dependent variable:

Diagnostics:

Regression:


Replace in selection:

    Select text region with mouse or keyboard.
    Ctrl+H to toggle find and replace
    Alt+L to toggle find in selection
    Ctrl+Alt+Enter to replace all (or enter to replace individually)


## Analysis plan
1. Import data
2. Research about approach
3. Clean Data
4. Feature Engineering
5. Feature Selection
6. Model training & tuning
7. Final model selection - evaluation on test data
8. Diagnostics


Additional : comparing simple train/test split forecast vs. rolling X-days ? Arima/Sarima/GARCH models vs linear / neural


### 1. Setting up environment

#### 1.1 Importing packages & setting-up parameters

In [24]:
# Set-up auto-reload functions for faster debugging 
# (automatically refreshes changes in subpackages codes)
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# Import parent directory (main project directory)
# for packages importing
import sys
import os

# Getting the parent directory name in which your script is running
parent = os.path.dirname(os.path.realpath(''))

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

# now we can import the module in the parent
# directory.

In [26]:
# Project packages import
import gp24package.data.make_dataset as gp24md
import gp24package.features.build_features as gp24feat
# import gp24package.explore_visualise.eda as gp23eda
# import gp24package.features.build_features as gp23feat
# import gp24package.models.hyperparameters_model as gp23hyperparam
# import gp24package.models.train_model as gp23train


# Pylance highligting package issue (not to be worried about)
# https://github.com/microsoft/pylance-release/blob/main/TROUBLESHOOTING.md#unresolved-import-warnings

# Standard Python libraries import
from IPython.display import display, HTML #  tidied-up display
from time import time #  project timer
from itertools import chain # for list iterations
from datetime import datetime
import pandas as pd
import numpy as np

# plots
import matplotlib.pyplot as plt
import seaborn as sn

# Necessary packages
import gp24package
import session_info # build and requirements.txt
import pickle # dump models

import ta #technical-analysis

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

#turning on plot display in JN
%matplotlib inline 
# Setting pandas display options
pd.options.display.max_columns = 300
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 250

In [27]:
# MANUAL_INPUT - mark of sections of code, that are not automated and have to be manually re-coded to new datasets.

# parameters
seed = 12345
target = 'SalePrice'
ID_vars = ['Order', 'PID']

# List of days - input to VarDescriptiveStatistics class
# based on list of contents we are creating new variables for last X days
# mean, std, min, max 
days = [3, 5, 30, 60, 90, 120, 240, 365]

# One Hot Encoding list
# contains varaibles nad possible values for one hot encoding
categories = [('month',[1,2,3,4,5,6,7,8,9,10,11,12]),
              ('weekday',[0,1,2,3,4,5,6])]

#### 1.2 Starting project timer and exporting requirements

In [28]:
# Starting project timer
tic_all = time()

In [29]:
# Collecting packages info and saving to requirements.txt file
session_info.show(cpu = True, std_lib = True, dependencies = True, write_req_file = True,
                  req_file_name = 'requirements.txt')

#### 1.3 Importing and inspecting source data

In [30]:
data = gp24md.MakeDataset('sphist.csv').data
print(type(data))
pd.options.display.float_format = '{:.4f}'.format
display(HTML(data.head().to_html()))

<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.4199,2090.4199,2066.7800,2077.0701,4043820000.0000,2077.0701
1,2015-12-04,2051.2400,2093.8401,2051.2400,2091.6899,4214910000.0000,2091.6899
2,2015-12-03,2080.7100,2085.0000,2042.3500,2049.6201,4306490000.0000,2049.6201
3,2015-12-02,2101.7100,2104.2700,2077.1101,2079.5100,3950640000.0000,2079.5100
4,2015-12-01,2082.9299,2103.3701,2082.9299,2102.6299,3712120000.0000,2102.6299


In [31]:
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(ascending = True, inplace  = True, by = 'Date')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 16590 entries, 16589 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.0 MB
None


In [32]:
data2 = gp24feat.DateFeatures(data_in = data, variable = 'Date').output()
data2.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000


In [33]:
# Using OneHotEncoder from sklearn to create binary variables representing months and weekdays
data3 = gp24feat.FeaturesOneHotEncoding(data_in = data2, features_list = categories).transform(data2)
data3.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
16589,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16588,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
16587,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
16586,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
16585,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [34]:
# Concatenate to original data
data4 = pd.concat([data2, data3], axis = 1)
data4.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [35]:
data5 = gp24feat.VarDescriptiveStatistics(data_in = data4, days = days, var_name = 'Close').output()
print(data5.shape)
display(HTML(data5.head().to_html()))

(16590, 576)


,Close_mean_3,Close_stdev_3,Close_max_3,Close_min_3,Close_mean_5,Close_stdev_5,Close_max_5,Close_min_5,Close_mean_30,Close_stdev_30,Close_max_30,Close_min_30,Close_mean_60,Close_stdev_60,Close_max_60,Close_min_60,Close_mean_90,Close_stdev_90,Close_max_90,Close_min_90,Close_mean_120,Close_stdev_120,Close_max_120,Close_min_120,Close_mean_240,Close_stdev_240,Close_max_240,Close_min_240,Close_mean_365,Close_stdev_365,Close_max_365,Close_min_365,Close_mean_3_stdev_3_ratio,Close_mean_3_max_3_ratio,Close_mean_3_min_3_ratio,Close_stdev_3_mean_3_ratio,Close_stdev_3_max_3_ratio,Close_stdev_3_min_3_ratio,Close_max_3_mean_3_ratio,Close_max_3_stdev_3_ratio,Close_max_3_min_3_ratio,Close_min_3_mean_3_ratio,Close_min_3_stdev_3_ratio,Close_min_3_max_3_ratio,Close_mean_3_stdev_5_ratio,Close_mean_3_max_5_ratio,Close_mean_3_min_5_ratio,Close_stdev_3_mean_5_ratio,Close_stdev_3_max_5_ratio,Close_stdev_3_min_5_ratio,Close_max_3_mean_5_ratio,Close_max_3_stdev_5_ratio,Close_max_3_min_5_ratio,Close_min_3_mean_5_ratio,Close_min_3_stdev_5_ratio,Close_min_3_max_5_ratio,Close_mean_3_mean_5_ratio,Close_stdev_3_stdev_5_ratio,Close_max_3_max_5_ratio,Close_min_3_min_5_ratio,Close_mean_3_stdev_30_ratio,Close_mean_3_max_30_ratio,Close_mean_3_min_30_ratio,Close_stdev_3_mean_30_ratio,Close_stdev_3_max_30_ratio,Close_stdev_3_min_30_ratio,Close_max_3_mean_30_ratio,Close_max_3_stdev_30_ratio,Close_max_3_min_30_ratio,Close_min_3_mean_30_ratio,Close_min_3_stdev_30_ratio,Close_min_3_max_30_ratio,Close_mean_3_mean_30_ratio,Close_stdev_3_stdev_30_ratio,Close_max_3_max_30_ratio,Close_min_3_min_30_ratio,Close_mean_3_stdev_60_ratio,Close_mean_3_max_60_ratio,Close_mean_3_min_60_ratio,Close_stdev_3_mean_60_ratio,Close_stdev_3_max_60_ratio,Close_stdev_3_min_60_ratio,Close_max_3_mean_60_ratio,Close_max_3_stdev_60_ratio,Close_max_3_min_60_ratio,Close_min_3_mean_60_ratio,Close_min_3_stdev_60_ratio,Close_min_3_max_60_ratio,Close_mean_3_mean_60_ratio,Close_stdev_3_stdev_60_ratio,Close_max_3_max_60_ratio,Close_min_3_min_60_ratio,Close_mean_3_stdev_90_ratio,Close_mean_3_max_90_ratio,Close_mean_3_min_90_ratio,Close_stdev_3_mean_90_ratio,Close_stdev_3_max_90_ratio,Close_stdev_3_min_90_ratio,Close_max_3_mean_90_ratio,Close_max_3_stdev_90_ratio,Close_max_3_min_90_ratio,Close_min_3_mean_90_ratio,Close_min_3_stdev_90_ratio,Close_min_3_max_90_ratio,Close_mean_3_mean_90_ratio,Close_stdev_3_stdev_90_ratio,Close_max_3_max_90_ratio,Close_min_3_min_90_ratio,Close_mean_3_stdev_120_ratio,Close_mean_3_max_120_ratio,Close_mean_3_min_120_ratio,Close_stdev_3_mean_120_ratio,Close_stdev_3_max_120_ratio,Close_stdev_3_min_120_ratio,Close_max_3_mean_120_ratio,Close_max_3_stdev_120_ratio,Close_max_3_min_120_ratio,Close_min_3_mean_120_ratio,Close_min_3_stdev_120_ratio,Close_min_3_max_120_ratio,Close_mean_3_mean_120_ratio,Close_stdev_3_stdev_120_ratio,Close_max_3_max_120_ratio,Close_min_3_min_120_ratio,Close_mean_3_stdev_240_ratio,Close_mean_3_max_240_ratio,Close_mean_3_min_240_ratio,Close_stdev_3_mean_240_ratio,Close_stdev_3_max_240_ratio,Close_stdev_3_min_240_ratio,Close_max_3_mean_240_ratio,Close_max_3_stdev_240_ratio,Close_max_3_min_240_ratio,Close_min_3_mean_240_ratio,Close_min_3_stdev_240_ratio,Close_min_3_max_240_ratio,Close_mean_3_mean_240_ratio,Close_stdev_3_stdev_240_ratio,Close_max_3_max_240_ratio,Close_min_3_min_240_ratio,Close_mean_3_stdev_365_ratio,Close_mean_3_max_365_ratio,Close_mean_3_min_365_ratio,Close_stdev_3_mean_365_ratio,Close_stdev_3_max_365_ratio,Close_stdev_3_min_365_ratio,Close_max_3_mean_365_ratio,Close_max_3_stdev_365_ratio,Close_max_3_min_365_ratio,Close_min_3_mean_365_ratio,Close_min_3_stdev_365_ratio,Close_min_3_max_365_ratio,Close_mean_3_mean_365_ratio,Close_stdev_3_stdev_365_ratio,Close_max_3_max_365_ratio,Close_min_3_min_365_ratio,Close_mean_5_stdev_5_ratio,Close_mean_5_max_5_ratio,Close_mean_5_min_5_ratio,Close_stdev_5_mean_5_ratio,Close_stdev_5_max_5_ratio,Close_stdev_5_min_5_ratio,Close_max_5_mean_5_ratio,Close_max_5_stdev_5_ratio,Close_max_5_min_5_ratio,Close_min_5_mean_5

In [36]:
data4["DailyReturn"] = data4["Close"].pct_change(periods = 1)
data6 = gp24feat.VarDescriptiveStatistics(data_in = data4, days = days, var_name = 'DailyReturn').output()
print(data6.shape)
display(HTML(data6.head().to_html()))

(16590, 576)


,DailyReturn_mean_3,DailyReturn_stdev_3,DailyReturn_max_3,DailyReturn_min_3,DailyReturn_mean_5,DailyReturn_stdev_5,DailyReturn_max_5,DailyReturn_min_5,DailyReturn_mean_30,DailyReturn_stdev_30,DailyReturn_max_30,DailyReturn_min_30,DailyReturn_mean_60,DailyReturn_stdev_60,DailyReturn_max_60,DailyReturn_min_60,DailyReturn_mean_90,DailyReturn_stdev_90,DailyReturn_max_90,DailyReturn_min_90,DailyReturn_mean_120,DailyReturn_stdev_120,DailyReturn_max_120,DailyReturn_min_120,DailyReturn_mean_240,DailyReturn_stdev_240,DailyReturn_max_240,DailyReturn_min_240,DailyReturn_mean_365,DailyReturn_stdev_365,DailyReturn_max_365,DailyReturn_min_365,DailyReturn_mean_3_stdev_3_ratio,DailyReturn_mean_3_max_3_ratio,DailyReturn_mean_3_min_3_ratio,DailyReturn_stdev_3_mean_3_ratio,DailyReturn_stdev_3_max_3_ratio,DailyReturn_stdev_3_min_3_ratio,DailyReturn_max_3_mean_3_ratio,DailyReturn_max_3_stdev_3_ratio,DailyReturn_max_3_min_3_ratio,DailyReturn_min_3_mean_3_ratio,DailyReturn_min_3_stdev_3_ratio,DailyReturn_min_3_max_3_ratio,DailyReturn_mean_3_stdev_5_ratio,DailyReturn_mean_3_max_5_ratio,DailyReturn_mean_3_min_5_ratio,DailyReturn_stdev_3_mean_5_ratio,DailyReturn_stdev_3_max_5_ratio,DailyReturn_stdev_3_min_5_ratio,DailyReturn_max_3_mean_5_ratio,DailyReturn_max_3_stdev_5_ratio,DailyReturn_max_3_min_5_ratio,DailyReturn_min_3_mean_5_ratio,DailyReturn_min_3_stdev_5_ratio,DailyReturn_min_3_max_5_ratio,DailyReturn_mean_3_mean_5_ratio,DailyReturn_stdev_3_stdev_5_ratio,DailyReturn_max_3_max_5_ratio,DailyReturn_min_3_min_5_ratio,DailyReturn_mean_3_stdev_30_ratio,DailyReturn_mean_3_max_30_ratio,DailyReturn_mean_3_min_30_ratio,DailyReturn_stdev_3_mean_30_ratio,DailyReturn_stdev_3_max_30_ratio,DailyReturn_stdev_3_min_30_ratio,DailyReturn_max_3_mean_30_ratio,DailyReturn_max_3_stdev_30_ratio,DailyReturn_max_3_min_30_ratio,DailyReturn_min_3_mean_30_ratio,DailyReturn_min_3_stdev_30_ratio,DailyReturn_min_3_max_30_ratio,DailyReturn_mean_3_mean_30_ratio,DailyReturn_stdev_3_stdev_30_ratio,DailyReturn_max_3_max_30_ratio,DailyReturn_min_3_min_30_ratio,DailyReturn_mean_3_stdev_60_ratio,DailyReturn_mean_3_max_60_ratio,DailyReturn_mean_3_min_60_ratio,DailyReturn_stdev_3_mean_60_ratio,DailyReturn_stdev_3_max_60_ratio,DailyReturn_stdev_3_min_60_ratio,DailyReturn_max_3_mean_60_ratio,DailyReturn_max_3_stdev_60_ratio,DailyReturn_max_3_min_60_ratio,DailyReturn_min_3_mean_60_ratio,DailyReturn_min_3_stdev_60_ratio,DailyReturn_min_3_max_60_ratio,DailyReturn_mean_3_mean_60_ratio,DailyReturn_stdev_3_stdev_60_ratio,DailyReturn_max_3_max_60_ratio,DailyReturn_min_3_min_60_ratio,DailyReturn_mean_3_stdev_90_ratio,DailyReturn_mean_3_max_90_ratio,DailyReturn_mean_3_min_90_ratio,DailyReturn_stdev_3_mean_90_ratio,DailyReturn_stdev_3_max_90_ratio,DailyReturn_stdev_3_min_90_ratio,DailyReturn_max_3_mean_90_ratio,DailyReturn_max_3_stdev_90_ratio,DailyReturn_max_3_min_90_ratio,DailyReturn_min_3_mean_90_ratio,DailyReturn_min_3_stdev_90_ratio,DailyReturn_min_3_max_90_ratio,DailyReturn_mean_3_mean_90_ratio,DailyReturn_stdev_3_stdev_90_ratio,DailyReturn_max_3_max_90_ratio,DailyReturn_min_3_min_90_ratio,DailyReturn_mean_3_stdev_120_ratio,DailyReturn_mean_3_max_120_ratio,DailyReturn_mean_3_min_120_ratio,DailyReturn_stdev_3_mean_120_ratio,DailyReturn_stdev_3_max_120_ratio,DailyReturn_stdev_3_min_120_ratio,DailyReturn_max_3_mean_120_ratio,DailyReturn_max_3_stdev_120_ratio,DailyReturn_max_3_min_120_ratio,DailyReturn_min_3_mean_120_ratio,DailyReturn_min_3_stdev_120_ratio,DailyReturn_min_3_max_120_ratio,DailyReturn_mean_3_mean_120_ratio,DailyReturn_stdev_3_stdev_120_ratio,DailyReturn_max_3_max_120_ratio,DailyReturn_min_3_min_120_ratio,DailyReturn_mean_3_stdev_240_ratio,DailyReturn_mean_3_max_240_ratio,DailyReturn_mean_3_min_240_ratio,DailyReturn_stdev_3_mean_240_ratio,DailyReturn_stdev_3_max_240_ratio,DailyReturn_stdev_3_min_240_ratio,DailyReturn_max_3_mean_240_ratio,DailyReturn_max_3_stdev_240_ratio,DailyReturn_max_3_min_240_ratio,DailyReturn_min_3_mean_240_ratio,DailyReturn_min_3_stdev_240_ratio,DailyReturn_min

In [ ]:
CZUPURKACZ

# return_value = data[var_name].pct_change(periods = i)
# results[f'returns_{i}'] = return_value

In [ ]:
results = results.shift(periods=1)
print(results.shape)
results.head(15)

In [ ]:
# TRAIN TEST SPLIT

data['Date_split'] = np.where(data["Date"] > datetime(year=2015, month=4, day=1),1,0)

In [ ]:

# Create technical indicators

# https://www.investopedia.com/terms/s/sma.asp
data['SMA_5'] = ta.trend.sma_indicator(data['Close'], window=5)
data['SMA_5'] = data['SMA_5'].shift(periods=1)

data['SMA_10'] = ta.trend.sma_indicator(data['Close'], window=10)
data['SMA_10'] = data['SMA_10'].shift(periods=1)

data['SMA_30'] = ta.trend.sma_indicator(data['Close'], window=30)
data['SMA_30'] = data['SMA_30'].shift(periods=1)
# https://www.investopedia.com/terms/e/ema.asp
data['EMA_5'] = ta.trend.ema_indicator(data['Close'], window=5)
data['EMA_5'] = data['EMA_5'].shift(periods=1)

data['EMA_10'] = ta.trend.ema_indicator(data['Close'], window=10)
data['EMA_10'] = data['EMA_10'].shift(periods=1)

data['EMA_30'] = ta.trend.ema_indicator(data['Close'], window=30)
data['EMA_30'] = data['EMA_30'].shift(periods=1)

# https://www.investopedia.com/terms/r/rsi.asp
data['RSI_14'] = ta.momentum.rsi(data['Close'], window=14)
data['RSI_14'] = data['RSI_14'].shift(periods=1)


# https://www.investopedia.com/terms/p/pricerateofchange.asp
data['ROC_10'] = ta.momentum.roc(data['Close'], window=10)
data['ROC_10'] = data['ROC_10'].shift(periods=1)

# te funkcje zwracaja obiekty, a nie liczby - do sprawdzenia
data['AOI_34'] = ta.momentum.awesome_oscillator(high = data['High'], low = data['Low'])
data['AOI_34'] = data['AOI_34'].shift(periods=1)

data['KAMA_30'] = ta.momentum.kama(data['Close'])
data['KAMA_30'] = data['KAMA_30'].shift(periods=1)

data['PPO_30'] = ta.momentum.ppo(data['Close'])
data['PPO_30'] = data['PPO_30'].shift(periods=1)

data['SRSI_14'] = ta.momentum.stochrsi(data['Close'])
data['SRSI_14'] = data['SRSI_14'].shift(periods=1)

data['SO_14'] = ta.momentum.stoch(high = data['High'], low = data['Low'], close = data['Close'])
data['SO_14'] = data['SO_14'].shift(periods=1)

data['TSII_14'] = ta.momentum.tsi(data['Close'])
data['TSII_14'] = data['TSII_14'].shift(periods=1)

# Create seasonal indicators
data['Month'] = data['Date'].dt.month
data['DayOfWeek'] = data['Date'].dt.dayofweek


# Convert 'Month' and 'DayOfWeek' to cyclic coordinates
data['Month_sin'] = np.sin(2*np.pi*data['Month']/12)
data['Month_cos'] = np.cos(2*np.pi*data['Month']/12)
data['DayOfWeek_sin'] = np.sin(2*np.pi*data['DayOfWeek']/7)
data['DayOfWeek_cos'] = np.cos(2*np.pi*data['DayOfWeek']/7)


print(data.head(20))

            Date  Open  High   Low  Close     Volume  Adj Close  Date_split  \
16589 1950-01-03 16.66 16.66 16.66  16.66 1260000.00      16.66           0   
16588 1950-01-04 16.85 16.85 16.85  16.85 1890000.00      16.85           0   
16587 1950-01-05 16.93 16.93 16.93  16.93 2550000.00      16.93           0   
16586 1950-01-06 16.98 16.98 16.98  16.98 2010000.00      16.98           0   
16585 1950-01-09 17.08 17.08 17.08  17.08 2520000.00      17.08           0   
16584 1950-01-10 17.03 17.03 17.03  17.03 2160000.00      17.03           0   
16583 1950-01-11 17.09 17.09 17.09  17.09 2630000.00      17.09           0   
16582 1950-01-12 16.76 16.76 16.76  16.76 2970000.00      16.76           0   
16581 1950-01-13 16.67 16.67 16.67  16.67 3330000.00      16.67           0   
16580 1950-01-16 16.72 16.72 16.72  16.72 1460000.00      16.72           0   
16579 1950-01-17 16.86 16.86 16.86  16.86 1790000.00      16.86           0   
16578 1950-01-18 16.85 16.85 16.85  16.85 1570000.00